In [9]:
import numpy as np
import pandas as pd
import folium
import branca
import json

Avalanche accidents excel data is loaded into a dataframe.

It contains avalanche accidents location, elevation, aspect, danger level, number of people caught, buried and killed.

In [10]:
# Load avalanche accidents data
df_accidents = pd.read_excel('../accident_data/accidents_switzerland.xlsx')

In [11]:
df_accidents.head()

,Date,Canton,Starting zone,Elevation,Aspect,Activity,Danger level,caught,buried,killed
0,1997-10-26,BE,Kandersteg,3420,N,1,-,2,1.0,2
1,1997-12-19,GR,St. Moritz,2560,SW,2,3,2,1.0,1
2,1997-12-21,UR,Andermatt,2940,NW,3,4,2,1.0,1
3,1997-12-29,VS,Bagnes,2435,NE,1,3,2,2.0,1
4,1998-01-04,UR,Andermatt,2400,N,2,3,2,1.0,1


Data is processed in order to remove cities that do not match with google maps location and deal with issues concerning denger level.

In [12]:
# Data cleaning
df_accidents = df_accidents.drop([149,276],axis = 0)
df_accidents['Danger level'] = df_accidents['Danger level'].replace('-',np.nan)
df_accidents['Danger level'][df_accidents['Danger level'].str.contains('↗︎') == True] = float(df_accidents['Danger level'][0])
df_accidents['Danger level'] = df_accidents['Danger level'].abs()
df_accidents.head()

C:\Users\Arnaud\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Date,Canton,Starting zone,Elevation,Aspect,Activity,Danger level,caught,buried,killed
0,1997-10-26,BE,Kandersteg,3420,N,1,NaN,2,1.0,2
1,1997-12-19,GR,St. Moritz,2560,SW,2,3,2,1.0,1
2,1997-12-21,UR,Andermatt,2940,NW,3,4,2,1.0,1
3,1997-12-29,VS,Bagnes,2435,NE,1,3,2,2.0,1
4,1998-01-04,UR,Andermatt,2400,N,2,3,2,1.0,1


A map centered in Switzerland is created.

In [22]:
# Create map centered in Switzerland
switzerland = (46.875893, 8.289321)
m = folium.Map(location=switzerland, zoom_start=8, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ')

In order to locate each avalanche accident on the swiss map, we decided to use geopy in order to find the latitude and longitude correponding to each city name in the dataframe.

In [23]:
# Find coordinates for each avanlanche accident in Switzerland
coordinates = pd.read_excel('../accident_data/coordinates.xlsx')

# Coordinates are added to the dataframe for each accident
df_accidents = df_accidents.assign(Longitude = coordinates.Longitude, Latitude = coordinates.Latitude)
df_accidents.head()

,Date,Canton,Starting zone,Elevation,Aspect,Activity,Danger level,caught,buried,killed,Latitude,Longitude
0,1997-10-26,BE,Kandersteg,3420,N,1,NaN,2,1.0,2,46.469813,7.728972
1,1997-12-19,GR,St. Moritz,2560,SW,2,3,2,1.0,1,46.502976,9.772840
2,1997-12-21,UR,Andermatt,2940,NW,3,4,2,1.0,1,46.602754,8.612660
3,1997-12-29,VS,Bagnes,2435,NE,1,3,2,2.0,1,46.034050,7.199696
4,1998-01-04,UR,Andermatt,2400,N,2,3,2,1.0,1,46.617105,8.616889


For each accident, a marker is added on the map at the right location, with popups giving information about the starting zone, the elevation, the danger level, the number of people caught and killed.

In [24]:
import branca.colormap as cm

colorscale = cm.StepColormap(
    ['#CCFF66','#FFFF00','#FF9B00', '#FF0000', '#800080'],
    vmin=0., vmax=5.,
    index=[0, 1 ,2 ,3, 4, 5],
    caption="Avalanche Danger Level"
)


# iterrows
# add markers to the map
for index in df_accidents.index.values:
    pop_up = folium.Popup("<dl><dt>Starting zone: </dt>"
           + "<dd>" + df_accidents['Starting zone'][index] + "</dd>"
           + "<dt>Elevation:</dt>"
           + "<dd>" + str(df_accidents['Elevation'][index]) + "</dd>"
           + "<dt>Danger level: </dt>"
           + "<dd>" + str(df_accidents['Danger level'][index]) + "</dd>"
           + "<dt>Caught: </dt>"
           + "<dd>" + str(df_accidents['caught'][index]) + "</dd>"
           + "<dt>Killed: </dt>"
           + "<dd>" + str(df_accidents['killed'][index]) + "</dd>")
    
    if df_accidents['Danger level'][index] >= 1 :
        color_circle = colorscale(df_accidents['Danger level'][index])
    else:
        color_circle = '#808080'
    
    location = [df_accidents.Latitude[index],df_accidents.Longitude[index]]
    folium.CircleMarker(location, popup = pop_up, color = color_circle, radius=5).add_to(m)

The map is displayed.

In [25]:
m.add_child(colorscale)
m

In [57]:
# number of accidents for each danger level:
num_acc = df_accidents.groupby('Danger level').count()['Starting zone']
num_acc

Danger level
1      5
2    102
3    192
4     15
5      2
Name: Starting zone, dtype: int64